In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os

from compression import Region, Comparator

%matplotlib notebook

/home/angus/.local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Adjust to point to the location of the image files and executables

DATA_DIR = os.path.join(os.path.expanduser("~"), "data", "fits")
ZFP = "zfp"
SZ = "sz"
BPGENC = "bpgenc"
BPGDEC = "bpgdec"

# DATA_DIR = os.path.join(os.sep, "data", "users", os.environ["USER"])
# ZFP = os.path.join(os.path.expanduser("~"), "bin", "zfp")
# SZ = os.path.join(os.path.expanduser("~"), "bin", "sz")

TEMP_DIR = DATA_DIR

In [3]:
# These files don't have to exist, as no attempt is made to access them in this cell.
# For the sake of readability, the data directory path is prepended to all the filenames at the end. 

regions = {
    "GALFACTS N4 Q full": {
        "filename": "GALFACTS_N4_cube_Q_DQA3.3.1.fits"
    },

    "GALFACTS N4 Q 512x512": {
        "filename": "GALFACTS_N4_cube_Q_DQA3.3.1.fits", 
        "size": (512, 512),
        "centre": (768, 4600)
    },

    "GALFACTS N4 I full": {
        "filename": "GALFACTS_N4_cube_I_DQA3.3.1.fits"
    },

    "GALFACTS N4 I 1024x1024": {
        "filename": "ds9.fits"
    },
    
    "GALFACTS average I full": {
        "filename": "GALFACTS_S3_average_image_I.fits"
    },
    
    "GALFACTS average I 512x512": {
        "filename": "GALFACTS_S3_average_image_I.fits", 
        "size": (512, 512),
        "centre": (768, 4600)
    },
    
    "GALFACTS average I 1024x1024": {
        "filename": "DQA_GALFACTS_N4_0263_4023_avg_I.fits", 
        "size": (1024, 1024),
        "centre": (542, 4230)
    },

    "DEEP full": {
        "filename": "DEEP_5_IQU_cube.fits",
        "stokes": 0,
        "channel": 3
    },

    "DEEP 512x512": {
        "filename": "DEEP_5_IQU_cube.fits",
        "stokes": 0,
        "channel": 3,
        "size": (512, 512),
        "centre": (2048, 2048)
    },

    "supermosaic full": {
        "filename": "supermosaic.10.fits"
    },

    "supermosaic 1024x1024": {
        "filename": "supermosaic.10.fits",
        "size": (1024, 1024),
        "centre": (1000, 1000),
        "channel": 66
    },
    
    "DEEP cropped": {
        "filename": "cropped_512_512_10_3_DEEP_5_IQU.fits",
        "stokes": 0,
        "channel": 3
    },
    
    "m81": {
        "filename": "m81.fits"
    },
    
    "lmcq": {
        "filename": "LMCq.fits",
        "size": (1024, 1024),
        "centre": (4000, 4000)
    },
    "linmos": {
        "filename": "GMRT_EN1W.linmos.flatnoise.fits",
        "size": (1024, 1024),
        "centre": (6000, 6000)
    },
    "point9_dc1_noise0": {
        "filename": "point9_dc1_noise0.fits",
        "size": (256, 256),
        "centre": (1536, 1536)
    }
}

for name, region in regions.items():
    region["filename"] = os.path.join(DATA_DIR, region["filename"])

In [4]:
# Load the specified region from the image file
#region = Region.from_fits(**regions["GALFACTS average I 1024x1024"])
#region = Region.from_fits(**regions["lmcq"])
#region = Region.from_fits(**regions["supermosaic 1024x1024"])
#region = Region.from_fits(**regions["linmos"])
region = Region.from_fits(**regions["point9_dc1_noise0"])

In [5]:
# This may take a long time for large images. All possible error values are calculated and cached.

comparator = Comparator.compare_algorithms(
    region,
    colourmap=plt.cm.gray,
    logarithmic=False, # change to True to make the colourmap logarithmic
    zfp=ZFP,
    sz=SZ,
    bpgenc=BPGENC,
    bpgdec=BPGDEC,
    temp_dir=TEMP_DIR,
    nan_interpolation_method=None, # "linear", "cubic" or None
    bpg_quant_step=4, # step size for BPG quantizer
    tile_size=256 # images are tiled before compression and reassembled afterwards 
)

vmin: {}; vmax: {} 1.0 10.0
Running algorithm ZFP (Fixed precision) with parameter 4...
Running algorithm ZFP (Fixed precision) with parameter 5...
Running algorithm ZFP (Fixed precision) with parameter 6...
Running algorithm ZFP (Fixed precision) with parameter 7...
Running algorithm ZFP (Fixed precision) with parameter 8...
Running algorithm ZFP (Fixed precision) with parameter 9...
Running algorithm ZFP (Fixed precision) with parameter 10...
Running algorithm ZFP (Fixed precision) with parameter 11...
Running algorithm ZFP (Fixed precision) with parameter 12...
Running algorithm ZFP (Fixed precision) with parameter 13...
Running algorithm ZFP (Fixed precision) with parameter 14...
Running algorithm ZFP (Fixed precision) with parameter 15...
Running algorithm ZFP (Fixed precision) with parameter 16...
Running algorithm ZFP (Fixed precision) with parameter 17...
Running algorithm ZFP (Fixed precision) with parameter 18...
Running algorithm ZFP (Fixed precision) with parameter 19...
Ru

In [6]:
# This plots various types of errors.
# Cached values are used, so changing plot parameters should be fast.

comparator.widget_plots();

interactive(children=(SelectMultiple(description='Plots', index=(0, 1), options=('mean', 'max', 'median'), val…

In [ ]:
#comparator2 = Comparator.compare_algorithms(
#    region,
#    colourmap=plt.cm.viridis,
#    logarithmic=False, # change to True to make the colourmap logarithmic
#    zfp=ZFP,
#    sz=SZ,
#    temp_dir=TEMP_DIR,
#    nan_interpolation_method="cubic" # "linear", "cubic" or None
#)
#comparator2.widget_plots();

In [ ]:
# This shows images of a similar compression size compressed with various algorithms.
# Compressed images are not stored, so they must be recalculated.

# A blank plot indicates that the selected algorithm produced no images below the selected compressed size limit.

comparator.widget_images();